In [7]:
import pandas as pd 
import numpy as np 
from scipy.fft import fft, fftfreq
from scipy import signal
import matplotlib.pyplot as plt
import uuid
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
#from os import listdir
import os

%matplotlib inline


In [18]:
def clean1(filename): 
    data=pd.read_csv(filename,skiprows=4)
    data.columns=data.columns.str.strip()
    return data

def clean2(filename):
    data=pd.read_csv(filename, sep=",", header=None,skiprows=5).values
    
    
    return data

In [91]:
def get_data():
    arr = os.listdir('/Users/brysonpelechaty/Desktop/Capstone/Data')
    arr1=[]
    
    for file in os.listdir('/Users/brysonpelechaty/Desktop/Capstone/Data'):
        if file.endswith(".txt"):
            #print(os.path.join('/Users/brysonpelechaty/Desktop/Capstone/Data', file))
            arr1.append(os.path.join('/Users/brysonpelechaty/Desktop/Capstone/Data', file))
            
    for i in range(len(arr1)):
        data=clean1(arr1[i])
        
    data.head()
    
    return arr1
        #data=clean2(temp)
    

In [87]:
def search_string(filename):
    
    

In [96]:
data_list=get_data()

In [90]:
data.head()

,Sample Index,EXG Channel 0,EXG Channel 1,EXG Channel 2,EXG Channel 3,EXG Channel 4,EXG Channel 5,EXG Channel 6,EXG Channel 7,Accel Channel 0,...,Other.3,Other.4,Other.5,Other.6,Analog Channel 0,Analog Channel 1,Analog Channel 2,Timestamp,Other.7,Timestamp (Formatted)
0,125.0,52642.806189,-187500.022352,-25888.930367,-187500.022352,-187500.022352,-9827.123264,-187500.022352,-187500.022352,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.656970e+09,0.0,2022-07-04 15:24:16.860
1,126.0,53177.079937,-187500.022352,-26240.746825,-187500.022352,-187500.022352,-8556.046612,-187500.022352,-187500.022352,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.656970e+09,0.0,2022-07-04 15:24:16.860
2,127.0,56016.578497,-187500.022352,-21827.864269,-187500.022352,-187500.022352,-7967.882808,-187500.022352,-187500.022352,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.656970e+09,0.0,2022-07-04 15:24:16.860
3,128.0,55841.698449,-187500.022352,-20993.674814,-187500.022352,-187500.022352,-9053.998775,-187500.022352,-187500.022352,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.656970e+09,0.0,2022-07-04 15:24:16.860
4,129.0,53074.776003,-187500.022352,-25059.792407,-187500.022352,-187500.022352,-9866.462334,-187500.022352,-187500.022352,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.656970e+09,0.0,2022-07-04 15:24:16.860


In [46]:
clean2('/Users/brysonpelechaty/Desktop/Capstone/Data/greatgatsby.txt')

array([[83.0, 73891.06737268773, 60169.86789344168, ...,
        1656968380.234831, 0.0, ' 2022-07-04 14:59:40.234'],
       [84.0, 67289.61375827952, 58152.46649413902, ...,
        1656968380.234849, 0.0, ' 2022-07-04 14:59:40.234'],
       [85.0, 63388.92142640607, 57035.0133818404, ...,
        1656968380.2348561, 0.0, ' 2022-07-04 14:59:40.234'],
       ...,
       [64.0, 83856.65969331977, 68645.82820484975, ...,
        1656968897.857451, 0.0, ' 2022-07-04 15:08:17.857'],
       [65.0, 83502.1610262586, 67901.53746623248, ...,
        1656968897.865218, 0.0, ' 2022-07-04 15:08:17.865'],
       [66.0, 77081.89065240511, 64359.85885380016, ...,
        1656968897.865255, 0.0, ' 2022-07-04 15:08:17.865']], dtype=object)

In [4]:
fs = 1000
band = (15,50)

In [5]:
def notch(val, data, fs= 200):
    notch_freq_Hz = np.array([float(val)])
    for freq_Hz in np.nditer(notch_freq_Hz):
        bp_stop_Hz = freq_Hz + 3.0 * np.array([-1, 1])
        b, a = signal.butter(3, bp_stop_Hz / (fs / 2.0), 'bandstop')
        fin = data = signal.lfilter(b, a, data)
    return fin

In [6]:
def bandpass(start, stop, data, fs = 200):
    bp_Hz = np.array([start, stop])
    b, a = signal.butter(5, bp_Hz / (fs / 2.0), btype='bandpass')
    return signal.lfilter(b, a, data, axis=0)

In [7]:
def fft(data, fs):
    L = len(data)
    freq = np.linspace(0.0, 1.0 / (2.0 * fs **-1), L // 2)
    yi = np.fft.fft(data)[1:]
    y = yi[range(int(L / 2))]
    return freq, abs(y)

In [4]:
def fft_filter(data):
    
    channels = []
    
    bandpass_channels = []
    bandpass_notch_channels = []
    notch_channels = [] 
    fs = 1000
    band = (15,50)
    
    for i in range(8):
        channels.append(data[:,1+i].astype(np.float))
    t = len(channels[0])/fs
    time = np.linspace(0, t, len(channels[0])) 
   
    
    
    
    for i in range(len(channels)):
        notch_channels.append(notch(60,channels[i], fs = fs))
    
    #for i in range(len(notch_channels)):
        #plt.plot(time, notch_channels[i])
    
    
    for i in range(len(channels)):
        bandpass_channels.append(bandpass(band[0],band[1],channels[i], fs = fs))
    
    for i in range(len(notch_channels)):
        bandpass_notch_channels.append(bandpass(band[0],band[1],notch_channels[i], fs = fs)) 
        
        
    
    #for i in range(len(bandpass_notch_channels)):
     #   freq, y = fft(bandpass_notch_channels[i], fs)
      #  plt.plot(freq, y)
       # plt.ylabel("Magnintude of frequency")
       # plt.xlabel("Frequency in Hertz")
       # plt.title(title)
    #plt.ylim(0, 1e7)
    #plt.xlim(10,60) 
    
    
    return bandpass_notch_channels
        
    
    
    
    


    
    
    
    


In [3]:
def get_dataframes(data):
    bandpass_notch_channels=data
    df1=pd.DataFrame(columns=["Frequency","y"])
    df2=pd.DataFrame(columns=["Frequency","y"])
    df3=pd.DataFrame(columns=["Frequency","y"])
    df4=pd.DataFrame(columns=["Frequency","y"])
    df5=pd.DataFrame(columns=["Frequency","y"])
    df6=pd.DataFrame(columns=["Frequency","y"])
    df7=pd.DataFrame(columns=["Frequency","y"])
    df8=pd.DataFrame(columns=["Frequency","y"])
    for i in range(len(bandpass_notch_channels)):
        freq, y = fft(bandpass_notch_channels[i], fs)
        if(i==0):
            df1["Frequency"]=freq
            df1["y"]=y
        elif(i==1):
            df2["Frequency"]=freq
            df2["y"]=y
        elif(i==2):
            df3["Frequency"]=freq
            df3["y"]=y
        elif(i==3):
            df4["Frequency"]=freq
            df4["y"]=y
        elif(i==4):
            df5["Frequency"]=freq
            df5["y"]=y
        elif(i==5):
            df6["Frequency"]=freq
            df6["y"]=y
        elif(i==6):
            df7["Frequency"]=freq
            df7["y"]=y
        elif(i==7):
            df8["Frequency"]=freq
            df8["y"]=y
    
    return df1, df2, df3, df4, df5, df6, df7, df8

In [97]:
def make_vector(df1,df2,df3,df4,df5,df6,df7,df8,exp): 
    
    vector=(get_freq_band(df1))
    sensor1=get_freq_band(df1)
    vector=vector.append(get_freq_band(df2))
    sensor2=get_freq_band(df2)
    vector=vector.append(get_freq_band(df3))
    vector=vector.append(get_freq_band(df4))
    vector=vector.append(get_freq_band(df5))
    vector=vector.append(get_freq_band(df6))
    vector=vector.append(get_freq_band(df7))
    vector=vector.append(get_freq_band(df8))
    
    
    #print(vector)
    #print("vector length is ",len(vector))
    
    sensorno=[]
    sensor="sensor"
    band=1
    counter=0
    
    for i in range(40):
        sensorno.append(sensor+str(band))
        counter+=1
        if(counter==5):
            band+=1
            counter=0
    vector["Sensor"]=sensorno
    
    exp_no=[]
    
    for i in range(40):
        exp_no.append(exp) 
        
    vector["Label"]=exp_no
        
    
    values=[]
    
    for i in range(len(vector)):
        item=vector.iloc[i]["val"]
        values.append(item)
        
    #bar1=sensor1.plot.bar(x='band', y='val', legend=False)
    #bar2=sensor2.plot.bar(x="band", y="val",legend=False)
    
    vector.head()
    
    return vector

In [ ]:
def get_freq_band(data): 
    fft_vals =data["y"]

# Get frequencies for amplitudes in Hz
    fft_freq = data["Frequency"]

# Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

# Take the mean of the fft amplitude for each EEG band
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(fft_vals[freq_ix])

    # Plot the data (using pandas here cause it's easy)
    import pandas as pd
    df = pd.DataFrame(columns=['band', 'val'])
    df['band'] = eeg_bands.keys()
    df['val'] = [eeg_band_fft[band] for band in eeg_bands]
   # ax = df.plot.bar(x='band', y='val', legend=False)
   # ax.set_xlabel("EEG band")
   #ax.set_ylabel("Mean band Amplitude")
    #print(df)
    
    return df

In [102]:
def  do_experiment(data,expno):
    
    data=fft_filter(data,"cool")
    
    df1, df2, df3, df4, df5, df6, df7, df8=get_dataframes(data)
    
    data=make_vector(df1, df2, df3, df4, df5, df6, df7, df8)
    
    exp=[]
    for i in range(40):
        exp.append(expno) 
        
    data["Label"]=exp
    
    return data
    
    
    
    

In [105]:
def do_all(): 
    
    data_list=get_data()
    
    main_data=pd.DataFrame
    
    for i in range(len(data_list)):
        
        main_data.append(do_experiment(clean2(data_list[i]),data_list[i])) 
        
    
    

6f455a65-ae1d-468d-a6ba-58bfef531870
